In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
open_file = open("../input/spaceship-titanic/train.csv")

train_df = pd.read_csv(open_file)

train_df.head()


In [ ]:
train_df.describe()

In [ ]:
columns_to_drop = [] #I'm going to keep a list of the items that end up hot encoded

train_df.isnull().sum()

In [ ]:
train_df

There are a number of null values within most of the categories. I'm going to work on cleaning these up before I start looking at the model. 

In [ ]:
#starting with HomePlanet 

train_df["HomePlanet"].value_counts()

In [ ]:
train_df["HomePlanet"].fillna("Unknown Planet", inplace = True)

In [ ]:
planet_dummies = pd.get_dummies(train_df["HomePlanet"])

columns_to_drop.append("HomePlanet")

planet_dummies

In [ ]:
train_df = pd.concat([train_df, planet_dummies], axis = 1)

In [ ]:
train_df.head()

In [ ]:
train_df["CryoSleep"].isnull().sum()

In [ ]:
train_df["CryoSleep"].value_counts()

In [ ]:
(217)/(5439 + 3037)

In [ ]:
train_df["CryoSleep"].fillna(False, inplace = True) #since the missing values are only 2.56% of the cyrosleep column, I am going to just replace them with False. The primary value in that column

In [ ]:
train_df["CryoSleep"].replace([True, False], [1, 0], inplace = True)

Now onto the Cabin. Reading the backup for this dataset, it appears the meaninful part of this column will likely be what deck the passenger was on, and if they were starbord or port. 

In [ ]:
train_df["Cabin"].value_counts()

In [ ]:
#I want to seperate the individual decks the passengers are on 

train_df["Deck"] = train_df["Cabin"].astype(str).str[0]

In [ ]:
train_df["Deck"]

In [ ]:
train_df["Deck"].value_counts()

In [ ]:
#the percentage of the null values to values of the most frequent 
#value in the column are higher than 
#the CryoSleep column, therefore I am going to opt to make and "Unknown Deck" column
#via hot encoding

train_df["Deck"].replace("n", "Deck Unknown", inplace = True)

columns_to_drop.append("Deck")

deck_dummies = pd.get_dummies(train_df["Deck"])

train_df = pd.concat([train_df, deck_dummies], axis = 1)

In [ ]:
train_df.columns

In [ ]:
train_df["Side"] = train_df["Cabin"].astype(str).str[-1]

In [ ]:
train_df["Side"].value_counts()

In [ ]:
train_df["Side"].replace("n", "Side Unknown", inplace = True)

In [ ]:
side_dummies = pd.get_dummies(train_df['Side'])

columns_to_drop.append("Side")

side_dummies

In [ ]:
train_df = pd.concat([train_df, side_dummies],axis = 1)

In [ ]:
train_df.columns

In [ ]:
print(train_df["Destination"].isnull().sum())

train_df["Destination"].value_counts()

In [ ]:
train_df["Destination"].fillna("TRAPPIST-1e", inplace = True)

In [ ]:
dest_dummies = pd.get_dummies(train_df["Destination"])

columns_to_drop.append("Destination")

dest_dummies

In [ ]:
train_df = pd.concat([train_df, dest_dummies], axis = 1)

In [ ]:
vip_true = len(train_df[train_df["VIP"] == True])

vip_false = len(train_df[train_df["VIP"] == False])

vip_null = train_df["VIP"].isnull().sum()

print(vip_true)

print(vip_false)

print(vip_null)

print("null/false: " + str(vip_null/vip_false))

In [ ]:
train_df["VIP"].fillna(False, inplace = True)

train_df["VIP"].replace([True, False], [1, 0], inplace = True)

In [ ]:
train_df.columns

In [ ]:
age_med = train_df["Age"].median()

print(age_med)

train_df["Age"].fillna(age_med, inplace = True)

train_df["Age"].isnull().sum()

In [ ]:
train_df.isnull().sum()

In [ ]:
train_df["RoomService"].describe()

In [ ]:
train_df["RoomService"].fillna(0, inplace = True)

In [ ]:
train_df["FoodCourt"].describe()

In [ ]:
train_df["FoodCourt"].fillna(0, inplace = True)

In [ ]:
train_df["Spa"].describe()

In [ ]:
train_df["Spa"].fillna(0, inplace = True)

In [ ]:
train_df["VRDeck"].describe()

In [ ]:
train_df["VRDeck"].fillna(0, inplace = True)

In [ ]:
train_df["ShoppingMall"].describe()

In [ ]:
train_df["ShoppingMall"].fillna(0, inplace = True)

Now I have removed the null values from the data set. It will be important to review these actions after the results of the learning model are produced. 

In [ ]:
train_df["Name"].value_counts()

In [ ]:
train_df.isnull().sum()

In [ ]:
columns_to_drop.append("Name")
columns_to_drop.append("PassengerId")
columns_to_drop.append("Cabin")

train_df.drop(columns_to_drop, axis = 1, inplace = True)

In [ ]:
train_df.columns

In [ ]:
train_df.corr()

In [ ]:
train_df["CryoSleep"].value_counts()

In [ ]:
y = train_df["Transported"]

train_df.drop("Transported", axis = 1, inplace = True)

In [ ]:
from sklearn import tree

X = train_df

model = tree.DecisionTreeClassifier(criterion = "entropy", max_depth = 8)

model = model.fit(X, y)

print(model.get_depth())

print(model.get_n_leaves())

Now that we have trained our model, we need to test it. However, it will be necessary to transform the test data so that it reflects all the changes that the training data had. 

In [ ]:
open_file = open("../input/spaceship-titanic/test.csv")

test_df = pd.read_csv(open_file)

test_df_copy = test_df

In [ ]:
len(test_df)

In [ ]:
#planets 
test_df["HomePlanet"] = test_df["HomePlanet"].fillna("Unknown Planet")
planet_dummies = pd.get_dummies(test_df["HomePlanet"])
test_df = pd.concat([test_df, planet_dummies], axis = 1)

#CryoSleep
test_df["CryoSleep"] = test_df["CryoSleep"].fillna(False)
test_df["CryoSleep"] = test_df["CryoSleep"].replace([True, False], [1, 0])

#Deck
test_df["Deck"] = test_df["Cabin"].astype(str).str[0]
test_df["Deck"] = test_df["Deck"].replace("n", "Deck Unknown")
deck_dummies = pd.get_dummies(test_df["Deck"])
test_df = pd.concat([test_df, deck_dummies], axis = 1)

#Side
test_df["Side"] = test_df["Cabin"].astype(str).str[-1]
test_df["Side"] = test_df["Side"].replace("n", "Side Unknown")
side_dummies = pd.get_dummies(test_df['Side'])
test_df = pd.concat([test_df, side_dummies],axis = 1)

#Destination
test_df["Destination"] = test_df["Destination"].fillna("TRAPPIST-1e")
dest_dummies = pd.get_dummies(test_df["Destination"])
test_df = pd.concat([test_df, dest_dummies], axis = 1)

#VIP
test_df["VIP"] = test_df["VIP"].fillna(False)
test_df["VIP"] = test_df["VIP"].replace([True, False], [1, 0])


#Age
age_med = test_df["Age"].median()
test_df["Age"] = test_df["Age"].fillna(age_med)

#FoodCourt
test_df["FoodCourt"] = test_df["FoodCourt"].fillna(0)

#RoomService
test_df["RoomService"] = test_df["RoomService"].fillna(0)

#Spa
test_df["Spa"] = test_df["Spa"].fillna(0)

#VRDeck
test_df["VRDeck"] = test_df["VRDeck"].fillna(0)

#ShoppingMall
test_df["ShoppingMall"] = test_df["ShoppingMall"].fillna(0)

In [ ]:
test_df.isnull().sum()

In [ ]:
print(columns_to_drop)

In [ ]:
test_df = test_df.drop(columns_to_drop, axis = 1)

In [ ]:
test_df.columns

In [ ]:
X_test = test_df

predictions = model.predict(X_test)


In [ ]:
my_submission = pd.DataFrame({'PassengerId': test_df_copy["PassengerId"], "Transported": predictions})

my_submission = my_submission.set_index("PassengerId")

In [ ]:
my_submission

In [ ]:
my_submission.to_csv("submission.csv")